In [ ]:
# # # Import libraries
import numpy as np
import pandas as pd
import random
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler


import warnings
warnings.filterwarnings('ignore')


In [ ]:
# # #  import data
df = pd.read_csv('/content/drive/MyDrive/labtest/UNSW_noisy_data/UNSW_noisy_dur_05_6_5.csv')

NameError: name 'pd' is not defined

In [ ]:
# # #  Parameter configurations
# SIGMA = 0.5
# OUTLIER_PECENTAGE = 6
PERCENTAGE_REPAIRED = 1724*5

length = df.shape[0]

columns = df.columns.tolist()

# Shuffle data
df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
# # # # FID for repairing data
# def FID_repaired(working_list):
#     # working_list = x1

#     index = []
#     for i, j in enumerate(working_list):
#         if j == 'NaN':
#             index.append(i)

#     # count the number of NaN/compromised point
#     p1 = working_list.count('NaN')
#     # print(p1)

#     t = p1 # Total number of missing value

#     # Select the min & max from list
#     # working_list.remove('NaN')
#     count=0
#     for index_pos in index:
#         working_list.pop(index_pos-count)
#         count+=1

#     # find mean of all observed values
#     mean = np.mean(working_list)

#     #find min value
#     a = min(working_list)

#     #find max value
#     b = max(working_list)

#     # Calculate h = (b-a)/t
#     h = (b-a)/t

#     # Calculate the discrete universe U using u = (a + (s-1) x h + a + s x h)/2, s=1,2,3
#     U = []
#     for s in range(1,t+1):
#         u = (a + (s-1) * h + a + s * h)/2
#         U.append(u)

#     # print(U)

#     # Calculating the missing values
#     M = []
#     for u in U:
#         # print(U)

#         # Compute the contribution weight (micro) of each observed element x_i

#         contribution_weight_list = []

#         for i in working_list:
#             if abs(i-u) <= h:
#                 temp = 1-(abs(i-u)/h)
#             else:
#                 temp = 0
#             contribution_weight_list.append(temp)

#         # Calculate the sum of x_i to u1:
#         sum_contribution_weight_list = sum(contribution_weight_list)
#         # print(sum_contribution_weight_list)

#         # Calculate the contribution of an observed data x_i
#         sum_contribution_observed_data = []

#         for num1, num2 in zip(working_list, contribution_weight_list):
#         	sum_contribution_observed_data.append(num1 * num2)

#         sum_contribution_observed_data = sum(sum_contribution_observed_data)
#         # print(sum_contribution_observed_data)

#         # Calculate the missing values in x_i
#         if sum_contribution_weight_list == 0:
#             m = mean
#         else:
#             m = sum_contribution_observed_data/sum_contribution_weight_list

#         M.append(m)

#     # print('The values:',M)
#     # print('The index position:',index)
#     return [index,M]

In [ ]:
# # # # Repairing data
# data = df['dur'].to_list()

# # # # Determine the outlier for repairing
# index_outlier = []

# y = df['label2']
# count_outlier = 0
# for i, j in enumerate(y):
#     if j == 0.0:
#         index_outlier.append(i)
#         count_outlier+=1
#     if count_outlier == PERCENTAGE_REPAIRED: break

# # Determine the compromised data
# for ind in index_outlier:
#     data[ind] = 'NaN'

# # Recover compromised data
# working_list = data

# results = FID_repaired(working_list)

# # print('The recovered values:',results[1])
# # print('The index position:',results[0])

In [ ]:
# # Paper 1:

# # Implementation of AMSA-VAE method for missing data filling
# import numpy as np


# # Standard Scaling
# scaler = StandardScaler()
# data_scaled = scaler.fit_transform(df[['dur']])

# # Imputation
# imputer = KNNImputer(n_neighbors=5)
# data_imputed = imputer.fit_transform(data_scaled)

# # AMSA-VAE Model Definition
# input_layer = Input(shape=(1,))
# dense_1 = Dense(128, activation='relu')(input_layer)
# repeat_vector = RepeatVector(1)(dense_1)
# permute_layer = Permute((2, 1))(repeat_vector)
# attention_output = Attention()([permute_layer, permute_layer])
# flatten_attention = Reshape((128,))(attention_output)
# add_layer = Add()([dense_1, flatten_attention])
# output_layer = Dense(1)(add_layer)

# # Compile and Train the model
# model = Model(inputs=input_layer, outputs=output_layer)
# model.compile(optimizer='adam', loss='mse')
# model.fit(data_imputed, data_imputed, epochs=10, batch_size=16)

# # Predict and Fill Missing Data
# data_filled = model.predict(data_imputed)
# data_filled = scaler.inverse_transform(data_filled)
# df['RH'] = data_filled.flatten()  # Update the DataFrame with repaired data

In [ ]:
# # Paper 2:

# # # # Applying Switching Triple-Weight-SMOTE (NSS)
# # Step 1: Impute missing values using LNMF
# nmf_model = NMF(n_components=5, init='random', random_state=0)
# W = nmf_model.fit_transform(np.nan_to_num(df[['dur']]))
# H = nmf_model.components_
# df['RH'] = np.dot(W, H).flatten()  # Impute missing values and update the DataFrame

# # Step 2: Map to Empirical Feature Space (EFS) - simplified for demonstration
# scaler = StandardScaler()
# data_scaled = scaler.fit_transform(df[['dur']])
# kmeans = KMeans(n_clusters=2, random_state=0).fit(data_scaled)
# df['cluster'] = kmeans.labels_

# # Step 3: Apply Switching Triple-Weight-SMOTE
# # Assign synthetic samples according to cluster properties
# for cluster_label in df['cluster'].unique():
#     cluster_data = df[df['cluster'] == cluster_label]
#     cluster_center = cluster_data[['dur']].mean().values
#     distances, indices = pairwise_distances_argmin_min(cluster_data[['dur']], [cluster_center])
#     # Synthesize new samples if needed based on cluster distribution
#     if cluster_label == 0:  # Example condition
#         new_samples = cluster_data.sample(frac=0.1, replace=True)
#         df = pd.concat([df, new_samples])

# # Shuffle the final DataFrame
# df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
# Paper 3:

# AMSA-VAE implementation for data filling
class AMSA(Layer):
    def __init__(self, d_model, num_heads):
        super(AMSA, self).__init__()
        self.mha = MultiHeadAttention(num_heads=num_heads, key_dim=d_model)
        self.norm = LayerNormalization(epsilon=1e-6)
        self.dropout = Dropout(0.1)

    def call(self, x):
        # Expand dims to create the necessary 3D input (batch, seq_len, features)
        x = tf.expand_dims(x, axis=1)  # Expands to (batch, 1, features)
        attn_output = self.mha(x, x)  # Apply multi-head attention
        attn_output = self.dropout(attn_output)
        out1 = self.norm(x + attn_output)  # Add & normalize
        out1 = tf.squeeze(out1, axis=1)  # Squeeze back to 2D if necessary
        return out1

def create_amsa_vae(input_shape):
    inputs = Input(shape=input_shape)
    x = AMSA(d_model=128, num_heads=4)(inputs)
    x = Dense(64, activation='relu')(x)
    outputs = Dense(input_shape[0], activation='linear')(x)
    model = Model(inputs, outputs)
    return model

# Data scaling
scaler = StandardScaler()
data_scaled = scaler.fit_transform(df[['dur']])

# AMSA-VAE model setup and training
amsa_vae = create_amsa_vae((1,))
amsa_vae.compile(optimizer='adam', loss='mse')
amsa_vae.fit(data_scaled, data_scaled, epochs=10, batch_size=4)

# Predict and update filled data
data_filled = amsa_vae.predict(data_scaled)
df['RH'] = scaler.inverse_transform(data_filled).flatten()  # Update the DataFrame with repaired data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# # # Update the predicted data into dataset
pos = 0
for index_pos in results[0]:
  df_1 = df.iloc[index_pos]
  df_1['dur'] = results[1][pos]
  # df_1['label2'] = 1

  df.loc[index_pos] = df_1
  pos+=1


In [ ]:
# # # # Splitting the dataset
# y = df['label2']
# print(np.count_nonzero(y == 0))
# X = df.drop(['index', 'label','label2'], axis =1)
# print(np.count_nonzero(y == 0))

# # # New noisy dataset
df.to_csv('/content/drive/MyDrive/dataset/UNSW_missing_data/UNSW_missing_repaired_dur_01.csv')